In [15]:
import torch
import numpy as np
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [4]:
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)


In [5]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.4994, -0.1572],
        [ 1.7284,  0.6089],
        [ 0.4321,  0.7530],
        [-0.5623, -1.1505],
        [-0.2828, -0.3728],
        [-1.4344,  0.1905],
        [ 0.7387,  0.8707],
        [ 1.6102, -0.6496],
        [ 0.9396,  0.6427],
        [-1.1489,  1.1507]]) tensor([ 7.7400,  5.5960,  2.4993,  6.9884,  4.9046,  0.6894,  2.7116,  9.6483,
         3.8819, -2.0074])


In [8]:
class LinearReg(nn.Module):
    def __init__(self, n_features):
        super(LinearReg, self).__init__()
        self.linear = nn.Linear(n_features, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y
net = LinearReg(num_inputs)
print(net)


LinearReg(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [9]:
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

In [11]:
print(net)
print()
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

Linear(in_features=2, out_features=1, bias=True)


In [12]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2304, -0.3309]], requires_grad=True)
Parameter containing:
tensor([0.6373], requires_grad=True)


In [13]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) 

Parameter containing:
tensor([0.], requires_grad=True)

In [14]:
loss = nn.MSELoss()

In [16]:
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [19]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

In [20]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 8.926183
epoch 2, loss: 2.257545
epoch 3, loss: 0.876159


In [21]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.6084, -2.8668]], requires_grad=True)
4.2 Parameter containing:
tensor([3.5451], requires_grad=True)


In [ ]:
##